## Dashboard Creation

In [ ]:
#import dashboard_dataset to final_dataset data frame
final_dataset = pd.read_csv('./dashboard_dataset.csv')

In [ ]:
print(final_dataset.head(10))
print(final_dataset.shape)
print(final_dataset.info())

In [ ]:
#decoding year_num column and month_name column
final_dataset['year_num'] = final_dataset['year_num'].replace({0: '2013', 1: '2014', 2: '2015'})
final_dataset['month_name'] = final_dataset['month_name'].replace({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6:'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11:'November', 12: 'December'})

print(final_dataset.head(10))
print(final_dataset.tail(10))   

In [ ]:
#make data frame for the dashboard interactive
df_interactive = final_dataset.interactive()

In [ ]:
#creates menu button for the dashboard to select years
year_num_menu = pn.widgets.Select(name='Year', options=df_interactive['year_num'].unique().tolist(), value='2015')

updated_year_df = df_interactive[df_interactive['year_num'] == year_num_menu]

In [ ]:
#montly revenue
update_monthly_revenue = (updated_year_df.groupby('month_name')['revenue'].mean().to_frame().reset_index().sort_values(by='month_name').reset_index(drop=True))
update_monthly_revenue_plot = update_monthly_revenue.hvplot.bar(x='month_name', y='revenue', rot=90, title='Average Revenue Per Month by year')

In [ ]:
#monthly item count
update_monthly_item_count = (updated_year_df.groupby('month_name')['item_cnt_month'].mean().to_frame().reset_index().sort_values(by='month_name').reset_index(drop=True))
update_monthly_item_count_pie = update_monthly_item_count.plot(kind='pie', y='item_cnt_month', label='month_name', autopct='%1.1f%%', title='Average Item Count Per Month by year')

In [ ]:
#sales by category
update_sales_by_category = (updated_year_df.groupby('item_category_name')['revenue'].mean().to_frame().reset_index().sort_values(by='revenue').reset_index(drop=True))
update_sales_by_category_plot = update_sales_by_category.hvplot.area(x='item_category_name', y='revenue', rot=90, title='Average Revenue Per Category by year', height=500, width=1000)

In [ ]:
#sales by shop
update_sales_by_shop = (updated_year_df.groupby('shop_name')['revenue'].mean().to_frame().reset_index().sort_values(by='revenue').reset_index(drop=True))
update_sales_by_shop_plot = update_sales_by_shop.hvplot.area(x='shop_name', y='revenue', rot=90, title='Average Revenue Per Shop by year', height=500, width=1000)

In [ ]:
#best selling category in all year
update_best_selling_category = (df_interactive.groupby('item_category_name')['item_cnt_month'].mean().to_frame().reset_index().sort_values(by='item_cnt_month', ascending=False).reset_index(drop=True))
update_best_selling_category_plot = update_best_selling_category.hvplot.barh(x='item_category_name', y='item_cnt_month', rot=90, title='Average Item Count Per Category', height=1000, width=1000)

In [ ]:
#creates echarts bar for the dashboard to display revenue by year
revenue_by_year_bar = { 'title' : { 'text' : 'Revenue by Year' }, 'tooltip' : { 'trigger': 'axis' }, 'legend': { 'data': ['Revenue'] }, 'xAxis' : { 'data' : final_dataset['year_num'].unique().tolist() }, 'yAxis' : { }, 'series' : [{ 'name' : 'Revenue', 'type' : 'bar', 'data' : final_dataset.groupby('year_num')['revenue'].sum().tolist() }] }
revenue_by_year_echart_pane = pn.pane.ECharts(revenue_by_year_bar, width=400, height=400)

In [ ]:
template = pn.template.FastListTemplate(
    title='Grocerry Sales Dashboard', 
    sidebar=[pn.pane.Markdown("# Revenue by Year"),
                revenue_by_year_echart_pane, 
             pn.pane.Markdown("#### Select Year"), 
             year_num_menu],
    main=[pn.Row(pn.Column(update_monthly_revenue_plot.panel(width=500)), 
                pn.Column(update_monthly_item_count_pie.panel(width=500), margin=(0,25)), 
                 ), 
          pn.Row(update_sales_by_category_plot),
          pn.Row(update_sales_by_shop_plot),
          pn.Row(update_best_selling_category_plot)],

    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.show()